# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,854.048532,0.777579,1234,854.826111,853.270953,1709.652223,1706.541906
6,439.446936,0.112118,1234,439.559054,439.334817,879.118108,878.669635
7,202.557915,0.785917,1234,203.343832,201.771998,406.687665,403.543997
8,869.405574,0.054340,1234,869.459914,869.351234,1738.919827,1738.702467
9,45.842140,-0.519480,1234,45.322660,46.361621,90.645320,92.723241
10,145.592102,0.555468,1234,146.147570,145.036633,292.295141,290.073267
11,957.067574,-0.674139,1234,956.393435,957.741713,1912.786870,1915.483427
12,380.674236,0.628683,1234,381.302919,380.045553,762.605837,760.091105
13,221.707785,0.816067,1234,222.523852,220.891718,445.047704,441.783435
14,84.795770,0.770762,1234,85.566532,84.025008,171.133063,168.050017


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-854.826111,853.270953
6,-439.559054,439.334817
7,-203.343832,201.771998
8,-869.459914,869.351234
9,-45.322660,46.361621
10,-146.147570,145.036633
11,-956.393435,957.741713
12,-381.302919,380.045553
13,-222.523852,220.891718
14,-85.566532,84.025008


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-854.826111,853.270953
6,-439.559054,439.334817
7,-203.343832,201.771998
8,-869.459914,869.351234
9,-45.322660,46.361621
10,-146.147570,145.036633
11,-956.393435,957.741713
12,-381.302919,380.045553
13,-222.523852,220.891718
14,-85.566532,84.025008


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-854.82611128,  853.27095319],
       [-439.5590538 ,  439.33481738],
       [-203.34383232,  201.77199828],
       [-869.45991355,  869.35123354],
       [ -45.32266014,   46.36162064],
       [-146.14757025,  145.03663329],
       [-956.39343498,  957.74171344],
       [-381.30291872,  380.04555251],
       [-222.52385182,  220.89171763],
       [ -85.5665315 ,   84.02500827]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1707.652223,-1706.541906
6,877.118108,-878.669635
7,404.687665,-403.543997
8,1736.919827,-1738.702467
9,88.645320,-92.723241
10,290.295141,-290.073267
11,1910.786870,-1915.483427
12,760.605837,-760.091105
13,443.047704,-441.783435
14,169.133063,-168.050017


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [16]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [17]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [18]:
data['a']  = np.nan

In [19]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [20]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [21]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
